# 로또 당첨정보 크롤링

In [139]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time
import re 
import pandas as pd
import numpy as np

In [140]:
# 전체 리스트 
win_result_list = []
win_date_list = []
total_reward_list = []
each_reward_list = []
num_of_winner_list = []
win_cat_list = []
total_saleprice_list = []

# 크롬드라이버 옵션 지정 : 창 최대화
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# 드라이버 실행
driver = webdriver.Chrome('chromedriver.exe',options = options)

# 로또 당첨번호 확인 홈페이지 접속
driver.get('https://dhlottery.co.kr/gameResult.do?method=byWin')

# 회차목록 가져오기
round_list = driver.find_element_by_id('dwrNoList').get_attribute('innerText').split('\n')

# 각 회차별 당첨번호 가져오기
for r in round_list:
    
    # 드롭다운 회차 메뉴 값 변경
    select_round = Select(driver.find_element_by_id('dwrNoList')) 
    select_round.select_by_value(str(r))
    driver.find_element_by_id('searchBtn').click()
    
    # 대기시간 1초
    time.sleep(1)
    
    # 사이트의 url 정보(html)를 가져와 Parsing
    soup = BeautifulSoup(driver.page_source,'html.parser')
    # 회차별 당첨번호 리스트 생성 : 'span' 태그의 'ball'로 시작한는 클래스 text 추출
    win_result = [num.text for num in soup.find_all('span',re.compile('ball.*'))]
    
    win_date = '-'.join(re.compile('\d+').findall(soup.select('p.desc')[0].text))
    
    #당첨금액 리스트 생성
    rewards = [int(re.sub('[,원]','',x.text)) for  x in soup.select('tr > td.tar')]
    total_rewards = []
    each_rewards = []
    for idx, reward in enumerate(rewards):
        # 등수별 전체 당첨금액
        if idx%2==0:
            total_rewards.append(reward)
        # 등수별 1인당 당첨금액
        else :
            each_rewards.append(reward)
    # 당첨자수 
    num_of_winners = [x/y if x !=0 else 0 for x,y in zip(total_rewards,each_rewards)]
    
    # 총판매액
    total_saleprice = int(re.sub('[,원]','',soup.select_one('li > strong').text))
    
    # 1등 당첨 유형
    cat_list= re.compile('\w+\d+').findall(soup.select('td')[5].text)
    win_cats = list(map(lambda x: re.compile('[가-힣]+').search(x).group(),cat_list))
    values = list(map(lambda x: re.compile('\d+').search(x).group(),cat_list))
    # 모든 유형 없을경우 해당 유형 생성 및 값에 0 입력
    none_cats = [cat for cat in ['자동','수동','반자동'] if cat not in win_cats]
    for none_cat in none_cats:
        win_cats.append(none_cat)
        values.append(0)
    # 자동, 수동, 반자동 순으로 값 정렬
    cat_dict  = {cat : value for cat,value in zip(win_cats,values)}
    sorted_values = [int(cat_dict[cat]) for cat in ['자동','수동','반자동']] 
    
    print('*'*20,r,'회차','*'*20)
    print('당첨번호 :',win_result)
    print('당첨날짜 :',win_date)
    print('총 당첨금액 :', total_rewards)
    print('1명당 당첨금액 :',each_rewards)
    print('당첨자수 :',num_of_winners)
    print('판매금액 :',total_saleprice)
    print('유형별 1등 당첨자수 : ',sorted_values)
    print()
    
    # 전체 리스트에 저장
    win_result_list.append(win_result)
    win_date_list.append(win_date)
    total_reward_list.append(total_rewards)
    each_reward_list.append(each_rewards)
    num_of_winner_list.append(num_of_winners)
    total_saleprice_list.append(total_saleprice)
    win_cat_list.append(sorted_values)
    
    
    

******************** 994 회차 ********************
당첨번호 : ['1', '3', '8', '24', '27', '35', '28']
당첨날짜 : 2021-12-18
총 당첨금액 : [22338984756, 3723164130, 3723166037, 6896850000, 11083305000]
1명당 당첨금액 : [1861582063, 53188059, 1328279, 50000, 5000]
당첨자수 : [12.0, 70.0, 2803.0, 137937.0, 2216661.0]
판매금액 : 95530936000
유형별 1등 당첨자수 :  [8, 4, 0]

******************** 993 회차 ********************
당첨번호 : ['6', '14', '16', '18', '24', '42', '44']
당첨날짜 : 2021-12-11
총 당첨금액 : [23947182378, 3991197070, 3991197924, 6629150000, 11103100000]
1명당 당첨금액 : [3991197063, 57017101, 1460907, 50000, 5000]
당첨자수 : [6.0, 70.0, 2732.0, 132583.0, 2220620.0]
판매금액 : 99323653000
유형별 1등 당첨자수 :  [4, 1, 1]

******************** 992 회차 ********************
당첨번호 : ['12', '20', '26', '33', '44', '45', '24']
당첨날짜 : 2021-12-04
총 당첨금액 : [23843466756, 3973911180, 3973911228, 6215550000, 10526410000]
1명당 당첨금액 : [1986955563, 68515710, 1674636, 50000, 5000]
당첨자수 : [12.0, 58.0, 2373.0, 124311.0, 2105282.0]
판매금액 : 97066498000
유형별 1등 당첨자수 : 

# 크롤링 결과 확인

In [141]:
# 각 리스트 원소 개수 확인
print(len(round_list))
print(len(win_result_list))
print(len(win_date_list))
print(len(total_reward_list))
print(len(each_reward_list))
print(len(num_of_winner_list))
print(len(total_saleprice_list))
print(len(win_cat_list))

994
994
994
994
994
994
994
994


In [142]:
# 당첨번호 6개 + 보너스
win_result_list

[['1', '3', '8', '24', '27', '35', '28'],
 ['6', '14', '16', '18', '24', '42', '44'],
 ['12', '20', '26', '33', '44', '45', '24'],
 ['13', '18', '25', '31', '33', '44', '38'],
 ['2', '4', '25', '26', '36', '37', '28'],
 ['17', '18', '21', '27', '29', '33', '26'],
 ['2', '13', '20', '30', '31', '41', '27'],
 ['2', '4', '15', '23', '29', '38', '7'],
 ['7', '10', '16', '28', '41', '42', '40'],
 ['17', '21', '23', '30', '34', '44', '19'],
 ['3', '10', '23', '35', '36', '37', '18'],
 ['13', '23', '26', '31', '35', '43', '15'],
 ['5', '7', '13', '20', '21', '44', '33'],
 ['27', '36', '37', '41', '43', '45', '32'],
 ['3', '13', '16', '23', '24', '35', '14'],
 ['7', '11', '16', '21', '27', '33', '24'],
 ['1', '7', '15', '32', '34', '42', '8'],
 ['2', '9', '10', '14', '22', '44', '16'],
 ['4', '12', '14', '25', '35', '37', '2'],
 ['7', '8', '9', '17', '22', '24', '5'],
 ['1', '2', '11', '16', '39', '44', '32'],
 ['22', '26', '31', '37', '41', '42', '24'],
 ['3', '6', '17', '23', '37', '39', '26

In [143]:
# 당첨날짜
win_date_list

['2021-12-18',
 '2021-12-11',
 '2021-12-04',
 '2021-11-27',
 '2021-11-20',
 '2021-11-13',
 '2021-11-06',
 '2021-10-30',
 '2021-10-23',
 '2021-10-16',
 '2021-10-09',
 '2021-10-02',
 '2021-09-25',
 '2021-09-18',
 '2021-09-11',
 '2021-09-04',
 '2021-08-28',
 '2021-08-21',
 '2021-08-14',
 '2021-08-07',
 '2021-07-31',
 '2021-07-24',
 '2021-07-17',
 '2021-07-10',
 '2021-07-03',
 '2021-06-26',
 '2021-06-19',
 '2021-06-12',
 '2021-06-05',
 '2021-05-29',
 '2021-05-22',
 '2021-05-15',
 '2021-05-08',
 '2021-05-01',
 '2021-04-24',
 '2021-04-17',
 '2021-04-10',
 '2021-04-03',
 '2021-03-27',
 '2021-03-20',
 '2021-03-13',
 '2021-03-06',
 '2021-02-27',
 '2021-02-20',
 '2021-02-13',
 '2021-02-06',
 '2021-01-30',
 '2021-01-23',
 '2021-01-16',
 '2021-01-09',
 '2021-01-02',
 '2020-12-26',
 '2020-12-19',
 '2020-12-12',
 '2020-12-05',
 '2020-11-28',
 '2020-11-21',
 '2020-11-14',
 '2020-11-07',
 '2020-10-31',
 '2020-10-24',
 '2020-10-17',
 '2020-10-10',
 '2020-10-03',
 '2020-09-26',
 '2020-09-19',
 '2020-09-

In [144]:
# 총 당첨금액
total_reward_list

[[22338984756, 3723164130, 3723166037, 6896850000, 11083305000],
 [23947182378, 3991197070, 3991197924, 6629150000, 11103100000],
 [23843466756, 3973911180, 3973911228, 6215550000, 10526410000],
 [23233328256, 3872221430, 3872223242, 6903400000, 11131970000],
 [24361333878, 4060222320, 4060223416, 6455650000, 10846095000],
 [23307074252, 3884512401, 3884513152, 6394550000, 10792180000],
 [24106404375, 4017734073, 4017734675, 6466100000, 10695125000],
 [23787116250, 3964519405, 3964520840, 7000200000, 11418410000],
 [23752751250, 3958791900, 3958792614, 6694450000, 11155835000],
 [24347529750, 4057921638, 4057923049, 6326150000, 10616200000],
 [24171043876, 4028507328, 4028509144, 6406450000, 10746675000],
 [25034222250, 4172370384, 4172372160, 6589000000, 10981540000],
 [24189045376, 4031507619, 4031508432, 6960400000, 11459665000],
 [25907221509, 4317870310, 4317870584, 6831500000, 11448250000],
 [23866102505, 3977683780, 3977685113, 6787550000, 11187670000],
 [22489607252, 3748267891

In [145]:
# 1인당 당첨금액
each_reward_list

[[1861582063, 53188059, 1328279, 50000, 5000],
 [3991197063, 57017101, 1460907, 50000, 5000],
 [1986955563, 68515710, 1674636, 50000, 5000],
 [2904166032, 35202013, 1304218, 50000, 5000],
 [1740095277, 72503970, 1420652, 50000, 5000],
 [5826768563, 61658927, 1589408, 50000, 5000],
 [2678489375, 65864493, 1508725, 50000, 5000],
 [2378711625, 54308485, 1335755, 50000, 5000],
 [2375275125, 56554170, 1417398, 50000, 5000],
 [2434752975, 65450349, 1572229, 50000, 5000],
 [3453006268, 62945427, 1551814, 50000, 5000],
 [2503422225, 53491928, 1580444, 50000, 5000],
 [3023630672, 49771699, 1375472, 50000, 5000],
 [1992863193, 66428774, 1528991, 50000, 5000],
 [3409443215, 56824054, 1411027, 50000, 5000],
 [1606400518, 47446429, 1097911, 50000, 5000],
 [2373391388, 56509319, 1399736, 50000, 5000],
 [1669905911, 47517648, 1574959, 50000, 5000],
 [3243867215, 54847997, 1226747, 50000, 5000],
 [2440410375, 60010092, 1294419, 50000, 5000],
 [1317034523, 56539361, 1415629, 50000, 5000],
 [2912742750,

In [146]:
# 당첨자수
num_of_winner_list

[[12.0, 70.0, 2803.0, 137937.0, 2216661.0],
 [6.0, 70.0, 2732.0, 132583.0, 2220620.0],
 [12.0, 58.0, 2373.0, 124311.0, 2105282.0],
 [8.0, 110.0, 2969.0, 138068.0, 2226394.0],
 [14.0, 56.0, 2858.0, 129113.0, 2169219.0],
 [4.0, 63.0, 2444.0, 127891.0, 2158436.0],
 [9.0, 61.0, 2663.0, 129322.0, 2139025.0],
 [10.0, 73.0, 2968.0, 140004.0, 2283682.0],
 [10.0, 70.0, 2793.0, 133889.0, 2231167.0],
 [10.0, 62.0, 2581.0, 126523.0, 2123240.0],
 [7.0, 64.0, 2596.0, 128129.0, 2149335.0],
 [10.0, 78.0, 2640.0, 131780.0, 2196308.0],
 [8.0, 81.0, 2931.0, 139208.0, 2291933.0],
 [13.0, 65.0, 2824.0, 136630.0, 2289650.0],
 [7.0, 70.0, 2819.0, 135751.0, 2237534.0],
 [14.0, 79.0, 3414.0, 154980.0, 2403760.0],
 [10.0, 70.0, 2826.0, 133915.0, 2217326.0],
 [14.0, 82.0, 2474.0, 125781.0, 2097459.0],
 [7.0, 69.0, 3085.0, 142375.0, 2270249.0],
 [9.0, 61.0, 2828.0, 135604.0, 2212347.0],
 [17.0, 66.0, 2636.0, 129473.0, 2149319.0],
 [8.0, 42.0, 2257.0, 117757.0, 2003966.0],
 [20.0, 80.0, 2570.0, 126989.0, 2122398.0

In [147]:
# 총판매액
total_saleprice_list

[95530936000,
 99323653000,
 97066498000,
 98026282000,
 99567047000,
 96525658000,
 98606195000,
 100269530000,
 99041240000,
 98811446000,
 98762367000,
 101899006000,
 101344251000,
 105645424000,
 99593380000,
 99507886000,
 98855197000,
 95895844000,
 97492178000,
 94253719000,
 94146055000,
 93953872000,
 93916813000,
 94471924000,
 93268371000,
 96728751000,
 95924229000,
 94515565000,
 95640447000,
 96174650000,
 95549895000,
 95452215000,
 95956365000,
 97153309000,
 98784879000,
 97435835000,
 98635947000,
 96654698000,
 97065255000,
 99784736000,
 101455314000,
 99180891000,
 102355646000,
 102937006000,
 108870714000,
 100518909000,
 100519459000,
 99449916000,
 99380178000,
 94837109000,
 107181377000,
 100081432000,
 92731965000,
 94867600000,
 93920183000,
 93340002000,
 93452425000,
 93720454000,
 92246146000,
 93934898000,
 93028347000,
 93484851000,
 96049289000,
 95695802000,
 94518415000,
 92344620000,
 90818982000,
 91067752000,
 88372251000,
 87901067000,
 8770825

In [148]:
# 유형별 1등 당첨자수(자동, 수동, 반자동)
win_cat_list

[[8, 4, 0],
 [4, 1, 1],
 [9, 3, 0],
 [5, 3, 0],
 [11, 3, 0],
 [4, 0, 0],
 [8, 1, 0],
 [5, 4, 1],
 [7, 2, 1],
 [8, 2, 0],
 [7, 0, 0],
 [9, 1, 0],
 [3, 4, 1],
 [8, 4, 1],
 [5, 2, 0],
 [8, 4, 2],
 [5, 5, 0],
 [8, 6, 0],
 [4, 3, 0],
 [7, 2, 0],
 [11, 5, 1],
 [8, 0, 0],
 [16, 4, 0],
 [4, 1, 1],
 [7, 5, 2],
 [13, 6, 1],
 [8, 4, 1],
 [2, 2, 0],
 [9, 1, 0],
 [5, 1, 1],
 [8, 2, 0],
 [8, 7, 0],
 [8, 4, 0],
 [5, 4, 0],
 [9, 0, 1],
 [7, 1, 0],
 [11, 4, 0],
 [7, 4, 0],
 [4, 6, 1],
 [9, 2, 1],
 [8, 2, 0],
 [9, 5, 0],
 [4, 5, 0],
 [8, 5, 1],
 [4, 3, 1],
 [9, 0, 1],
 [3, 7, 1],
 [14, 2, 2],
 [6, 5, 0],
 [10, 3, 0],
 [7, 6, 0],
 [4, 3, 0],
 [4, 2, 0],
 [7, 8, 1],
 [3, 4, 1],
 [6, 7, 0],
 [10, 0, 0],
 [9, 2, 0],
 [10, 3, 1],
 [5, 8, 0],
 [3, 1, 0],
 [8, 0, 0],
 [5, 2, 0],
 [8, 0, 0],
 [8, 0, 0],
 [9, 6, 1],
 [3, 3, 1],
 [4, 1, 1],
 [7, 3, 0],
 [8, 3, 1],
 [8, 1, 0],
 [6, 2, 0],
 [3, 3, 0],
 [16, 1, 0],
 [4, 3, 0],
 [3, 1, 1],
 [10, 8, 0],
 [8, 2, 0],
 [5, 5, 1],
 [4, 2, 0],
 [7, 4, 0],
 [14, 2, 0],
 [7,

In [149]:
# 최종 데이터프레임 생성
lotto_df = pd.DataFrame({'회차' : round_list,
             '당첨번호1' : [num[0] for num in win_result_list],
              '당첨번호2' : [num[1] for num in win_result_list],
              '당첨번호3' : [num[2] for num in win_result_list],
              '당첨번호4' : [num[3] for num in win_result_list],
              '당첨번호5' : [num[4] for num in win_result_list],
              '당첨번호6' : [num[5] for num in win_result_list],
              '당첨번호_보너스' : [num[6] for num in win_result_list],
             '당첨날짜' : win_date_list,
             '총당첨금액_1등' : [total_reward[0] for total_reward in total_reward_list],
              '총당첨금액_2등' : [total_reward[1] for total_reward in total_reward_list],
              '총당첨금액_3등' : [total_reward[2] for total_reward in total_reward_list],
              '총당첨금액_4등' : [total_reward[3] for total_reward in total_reward_list],
              '총당첨금액_5등' : [total_reward[4] for total_reward in total_reward_list],
             '1인당첨금액_1등' : [each_reward[0] for each_reward in each_reward_list],
              '1인당첨금액_2등' : [each_reward[1] for each_reward in each_reward_list],
              '1인당첨금액_3등' : [each_reward[2] for each_reward in each_reward_list],
              '1인당첨금액_4등' : [each_reward[3] for each_reward in each_reward_list],
              '1인당첨금액_5등' : [each_reward[4] for each_reward in each_reward_list],
             '당첨자수_1등' : [num_of_winner[0] for num_of_winner in num_of_winner_list],
              '당첨자수_2등' : [num_of_winner[1] for num_of_winner in num_of_winner_list],
              '당첨자수_3등' : [num_of_winner[2] for num_of_winner in num_of_winner_list],
              '당첨자수_4등' : [num_of_winner[3] for num_of_winner in num_of_winner_list],
              '당첨자수_5등' : [num_of_winner[4] for num_of_winner in num_of_winner_list],
             '판매금액' : total_saleprice_list,
             '1등당첨유형_자동' : [win_cat[0] for win_cat in win_cat_list],
             '1등당첨유형_수동' : [win_cat[1] for win_cat in win_cat_list],
             '1등당첨유형_반자동' : [win_cat[2] for win_cat in win_cat_list]})

In [150]:
lotto_df.head()

,회차,당첨번호1,당첨번호2,당첨번호3,당첨번호4,당첨번호5,당첨번호6,당첨번호_보너스,당첨날짜,총당첨금액_1등,...,1인당첨금액_5등,당첨자수_1등,당첨자수_2등,당첨자수_3등,당첨자수_4등,당첨자수_5등,판매금액,1등당첨유형_자동,1등당첨유형_수동,1등당첨유형_반자동
0,994,1,3,8,24,27,35,28,2021-12-18,22338984756,...,5000,12.0,70.0,2803.0,137937.0,2216661.0,95530936000,8,4,0
1,993,6,14,16,18,24,42,44,2021-12-11,23947182378,...,5000,6.0,70.0,2732.0,132583.0,2220620.0,99323653000,4,1,1
2,992,12,20,26,33,44,45,24,2021-12-04,23843466756,...,5000,12.0,58.0,2373.0,124311.0,2105282.0,97066498000,9,3,0
3,991,13,18,25,31,33,44,38,2021-11-27,23233328256,...,5000,8.0,110.0,2969.0,138068.0,2226394.0,98026282000,5,3,0
4,990,2,4,25,26,36,37,28,2021-11-20,24361333878,...,5000,14.0,56.0,2858.0,129113.0,2169219.0,99567047000,11,3,0


In [151]:
# 데이터프레임 저장
lotto_df.to_csv('로또당첨정보.csv',encoding='cp949',index = False)

# 추가 : 1등 당첨 판매점 정보 크롤링

In [167]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome('chromedriver.exe',options = options)
driver.get('https://pyony.com/lotto/rounds/latest/')
time.sleep(1)

store_name_list = []
store_cat_list = []
store_add_list = []

for r in round_list:
    
    # 드롭다운 회차 메뉴 값 변경
    select_round = Select(driver.find_element_by_id('id_round'))
    select_round.select_by_visible_text(str(r))
    driver.find_element_by_id('btn_search').click()
    time.sleep(1)
    
    # url정보(html) Paring
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 당첨 판매점 정보 없는 회차 존재 
    # 1등 당첨자 존재하지만 판매점 정보 존재 X -> 코드 실행 중지(1 - 261회 미존재)
    try:
        # 판매점 정보 가져오기
        store_info_list = [x.text.strip() for x in soup.select('tbody')[1].select('td')]
    except:
        # 1등 당첨자가 없을 때 판매점 정보 존재 X -> 다음 회차로 넘어감
        if soup.select('tbody > tr')[0].select_one('td').text =='0':
            continue
        # 1등 당첨자 존재 할 때 판매점 정보 존재 X -> 코드 실행 중지(이전 회차 모두 존재 X)
        else:
            break
    # 판매점 이름
    store_name = store_info_list[0::3]
    # 판매점 당첨 유형
    store_cat = store_info_list[1::3]
    # 판매점 주소
    store_add = store_info_list[2::3]
    
    print('*'*20,r,'회차','*'*20)
    print('1등 판매점 :',','.join(store_name),end='\n\n')
    print('당첨 유형 :',','.join(store_cat),end='\n\n')
    print('주소 :',','.join(store_add),end='\n\n')
    
    # exist_round_list : 판매점 정보 존채 회차 리스트 
    exist_round_list.append(r)
    store_name_list.append(store_name)
    store_cat_list.append(store_cat)
    store_add_list.append(store_add)

******************** 994 회차 ********************
1등 판매점 : 개미식품,드림,팬레코드,드림마트,금곡로,복권나라,드림통신(구)굿타임24아산석정점,씨유 충효그린점,운수대통복권판매점,부강 돈벼락,GS25(양산혜인점),행복을주는가게

당첨 유형 : 수동,자동,자동,수동,자동,자동,수동,수동,자동,자동,자동,자동

주소 : 서울 동작구 노량진로 110 건설상가 115호,서울 서대문구 증가로 247 1층,부산 남구 수영로 253,인천 서구 검단로 490,경기 수원시 권선구 금곡로197번길 18-23 해피라움블루 105호(금곡동, 해피라움블루),충북 청주시 청원구 율봉로176번길 8-2,충남 아산시 충무로 230,경북 경주시 충효녹지길 52  CU충효그린점,경남 거제시 능포로 108 동헌하이츠 102동 109호,경남 양산시 물금로 41 양산물금양우내안애5차리버파크 상가108호 양우대산부동산 내,경남 양산시 웅상대로 955,(평산동),제주 서귀포시 태평로 505 가나노블휘닉스 1층 세븐일레븐

******************** 993 회차 ********************
1등 판매점 : 공원슈퍼,일등대박복권,영광로또,세븐일레븐아산용화성지점,위앤조이,GS25(경산정평점)

당첨 유형 : 자동,자동,자동,자동,반자동,수동

주소 : 경기 수원시 장안구 팔달로 257-1 1층(영화동),경기 화성시 효행로  1363,충북 충주시 충청대로 2220-2 1층,충남 아산시 시민로 292 성지빌딩 세븐일레븐 편의점내,충남 천안시 서북구 천안대로 1391,경북 경산시 강변서로51길 18

******************** 992 회차 ********************
1등 판매점 : 우일,로또,이마트24백석동문점,성공복권방,복권나라,해적책대여점 복권방,나래복권,황실복권방,로또복권판매도통점,나이스로또복권,삼삼마트,CU(통영북신에이스점)

당첨 유형 : 수동,자동,자동,수동,자동,자동,자동,자동,자동,자동,자동,수동

주소 : 서울 송파

In [185]:
# 최종 데이터프레임 생성
store_df = pd.DataFrame(columns = ['회차','판매점','당첨유형','주소'])

# 판매점 정보 존재 회차 수 만큼반복
for r in range(len(exist_round_list)):
    # 해당 회차에 해당하는 판매점 수 만큼 회차 원소값 복제
    r_list = [exist_round_list[r]] * len(store_name_list[r])
    # 회차별 데이터프레임 생성
    temp_df = pd.DataFrame({'회차' : r_list,
                          '판매점' : store_name_list[r],
                          '당첨유형' : store_cat_list[r],
                          '주소' : store_add_list[r]})
    #최종 데이터프레임에 회차별 데이터프레임 추가
    store_df = pd.concat([store_df,temp_df],axis=0)
store_df.reset_index(drop=True,inplace = True)

In [186]:
store_df.head(30)

,회차,판매점,당첨유형,주소
0,994,개미식품,수동,서울 동작구 노량진로 110 건설상가 115호
1,994,드림,자동,서울 서대문구 증가로 247 1층
2,994,팬레코드,자동,부산 남구 수영로 253
3,994,드림마트,수동,인천 서구 검단로 490
4,994,금곡로,자동,"경기 수원시 권선구 금곡로197번길 18-23 해피라움블루 105호(금곡동, 해피라..."
5,994,복권나라,자동,충북 청주시 청원구 율봉로176번길 8-2
6,994,드림통신(구)굿타임24아산석정점,수동,충남 아산시 충무로 230
7,994,씨유 충효그린점,수동,경북 경주시 충효녹지길 52 CU충효그린점
8,994,운수대통복권판매점,자동,경남 거제시 능포로 108 동헌하이츠 102동 109호
9,994,부강 돈벼락,자동,경남 양산시 물금로 41 양산물금양우내안애5차리버파크 상가108호 양우대산부동산 내


In [187]:
store_df.to_csv('판매점정보.csv',encoding='cp949',index = False)